# DEEP LEARNING 

# Partie sur les images

### Objectif à dépasser : weighted F1-score = 0.5534 (Resnet)
avec CNN/Transfer learning

Import libraries:

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
os.chdir('C:\\Users\\barry\\OneDrive - CSTBGroup\\image_ds\\images')

Récupérer les données:

In [2]:
X = pd.read_csv('X_train_update.csv',index_col=0)
y = pd.read_csv('Y_train_CVw08PX.csv',index_col=0)

Liste des images:

In [4]:
os.chdir('C:\\Users\\barry\\OneDrive - CSTBGroup\\image_ds')

In [5]:
import os #Miscellaneous operating system interfaces
#https://docs.python.org/3/library/os.html

#get current working directory
current_path = os.getcwd() 

#Training images path
images_path = current_path + r'/images/image_train/'

#List with the name of all training images
images_list = os.listdir(images_path)

Géneration nom des images:

In [6]:
#Create a column with the name of the picture
X['image name'] = 'image_' + X['imageid'].map(str) + '_product_' + X['productid'].map(str) + '.jpg'
X['image name']

0        image_1263597046_product_3804725264.jpg
1         image_1008141237_product_436067568.jpg
2          image_938777978_product_201115110.jpg
3           image_457047496_product_50418756.jpg
4         image_1077757786_product_278535884.jpg
                          ...                   
84911      image_941495734_product_206719094.jpg
84912    image_1188462883_product_3065095706.jpg
84913     image_1009325617_product_440707564.jpg
84914    image_1267353403_product_3942400296.jpg
84915       image_684671297_product_57203227.jpg
Name: image name, Length: 84916, dtype: object

### Répartition des images dans les échantillons train, validation et test

Répartion :

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = 123)

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = 123)

X_val = X_train.iloc[:2400,:]
y_val = y_train.iloc[:2400,:]

X_train = X_train.iloc[:21600,:]
y_train = y_train.iloc[:21600:,:]

X_test = X_test.iloc[:6000,:]
y_test = y_test.iloc[:6000,:]

Qualité de la répartitioon

trainy = pd.DataFrame(y_train.value_counts(), columns = ['Nombre_images_train'])
testy = pd.DataFrame(y_test.value_counts(), columns = ['Nombre_images_test'])
train_test = trainy.merge(testy, right_index = True, left_index = True)
train_test['y'] = train_test.index
yval = []
for i in train_test['y']:
    yval.append(i[0])
train_test['y'] = yval
train_test

# TRANSFER LEARNING

In [38]:
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator

train_data_generator = ImageDataGenerator(
        preprocessing_function = preprocess_input,  # data augmentation
        rotation_range = 10,
        width_shift_range = 0.1,
        height_shift_range = 0.1,
        zoom_range = 1.1,
        horizontal_flip = True
        )

val_data_generator = ImageDataGenerator(
        preprocessing_function = preprocess_input,  # data augmentation
        rotation_range = 10,
        width_shift_range = 0.1,
        height_shift_range = 0.1,
        zoom_range = 1.1,
        horizontal_flip = True
        )

test_data_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input)

In [42]:
X_train["class"] = y_train
X_test["class"] = y_test

In [43]:
X_val["class"] = y_val

In [45]:
batch_size = 32
path = os.chdir('C:\\Users\\barry\\OneDrive - CSTBGroup\\image_ds\\images\\image_train')
X_train["class"] = X_train["class"].astype(str)
X_test["class"] = X_test["class"].astype(str)
X_val["class"] = X_val["class"].astype(str)

train_generator = train_data_generator.flow_from_dataframe(dataframe=X_train,
                                                          directory=path,
                                                           x_col = "image name",
                                                           y_col = "class",
                                                           class_mode ="sparse",
                                                          target_size = (224, 224), 
                                                          batch_size = batch_size)


val_generator = val_data_generator.flow_from_dataframe(dataframe=X_val,
                                                          directory=path,
                                                           x_col = "image name",
                                                           y_col = "class",
                                                           class_mode ="sparse",
                                                          target_size = (224, 224), 
                                                          batch_size = batch_size)

#garde ou retire shuffle = False
test_generator = test_data_generator.flow_from_dataframe(dataframe=X_test,
                                                          directory=path,
                                                           x_col = "image name",
                                                           y_col = "class",
                                                           class_mode ="sparse",
                                                          target_size = (224, 224), 
                                                          batch_size = batch_size, shuffle = False)

Found 21600 validated image filenames belonging to 27 classes.
Found 2400 validated image filenames belonging to 27 classes.
Found 6000 validated image filenames belonging to 27 classes.


In [46]:
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras import backend as K
from keras.applications.vgg16 import VGG16

n_class = 27
# Modèle VGG16
base_model = VGG16(weights='imagenet', include_top=False) 
# Freezer les couches du VGG16
for layer in base_model.layers: 
    layer.trainable = False

model = Sequential()
model.add(base_model) # Ajout du modèle VGG16
model.add(GlobalAveragePooling2D()) 
model.add(Dense(1024,activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(n_class, activation='softmax'))

model.compile(optimizer=Adam(lr = 0.0001), loss='sparse_categorical_crossentropy', metrics=['acc'])

Résumé du modèle : 

In [47]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, None, None, 512)   14714688  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 27)               

Entrainement tous layers freezed : 

In [48]:
tl_2 = model.fit_generator(generator=train_generator, 
                                epochs = 2,
                                steps_per_epoch = len(X_train)//batch_size,
                                validation_data = val_generator,
                                validation_steps = len(X_val)//batch_size
                                )

C:\Users\barry\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2
675/675 [==============================] - 3511s 5s/step - loss: 3.3809 - acc: 0.1671 - val_loss: 2.3914 - val_acc: 0.3150
Epoch 2/2
675/675 [==============================] - 3513s 5s/step - loss: 2.5359 - acc: 0.2787 - val_loss: 2.2795 - val_acc: 0.3450


Entrainement 4 derniers layers freezed :

In [49]:
for layer in base_model.layers[-4:]:
    layer.trainable = True

In [50]:
model.compile(optimizer=Adam(lr=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
tl_2_defreeze = model.fit_generator(generator=train_generator, 
                                epochs = 2,
                                steps_per_epoch = len(X_train)//batch_size,
                                validation_data=test_generator,
                                validation_steps=len(X_val)//batch_size
                                )

Epoch 1/2
675/675 [==============================] - 3509s 5s/step - loss: 2.5078 - accuracy: 0.2879 - val_loss: 1.9700 - val_accuracy: 0.4321
Epoch 2/2
675/675 [==============================] - 3631s 5s/step - loss: 2.2307 - accuracy: 0.3554 - val_loss: 1.8522 - val_accuracy: 0.4617


In [51]:
y_tl_pred_2 = model.predict_generator(test_generator, verbose = 1)
y_tl_pred_2 = y_tl_pred_2.argmax(axis = 1)

C:\Users\barry\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


188/188 [==============================] - 717s 4s/step


In [56]:
from sklearn.metrics import f1_score
f1_score(test_generator.classes, y_tl_pred_2 , average = "weighted")

0.4375734213969084

In [60]:
from sklearn.metrics import confusion_matrix
print(pd.DataFrame(confusion_matrix( test_generator.classes, y_tl_pred_2)))

     0   1    2   3    4   5    6   7   8   9   ...  17   18  19   20  21  22  \
0   118   0    5   0    1   2    0   0   0   0  ...   0    4   2    1   0  20   
1     4  65    8   0   39   4    4   0   1  14  ...   1    0   0    8   0   0   
2     0   1  234   0    0   0    0   0   0   0  ...   1    1   0    0   0   1   
3     2  11    2   0    5   0    1   0   0   3  ...   1    2   0    2   0   2   
4     5  18    9   0  114   8   33   0  19  19  ...   3    9   0   16   0   3   
5     5   0   11   0   19  21    6   0   7   1  ...   0    9   0    3   0   0   
6     1   5    1   1    9   1  177   0  20  10  ...   6   11   1   51   0   0   
7     0   1    0   0    2   0    2   0   3  12  ...   0    8   0   10   0   1   
8     1   0    1   0   31   4   11   0  38  14  ...   0    5   6   19   1   0   
9     2   0    0   0   31   4    4   0   6  56  ...   0   10   1   21   1   1   
10    1   1    0   0   11   1    9   0   6  16  ...   1   14   2   21   1   0   
11    0   0    0   0    5   

### Features extraction (en cours)

In [58]:
from sklearn.svm import SVC

intermediate_layer_model = Model(model.input, model.layers[2].output)
X_train_features = intermediate_layer_model.predict(X_train)
X_test_features = intermediate_layer_model.predict(X_test)
svm = SVM()
svm.fit(X_train_features, Y_train)
svm.score(X_test_features, Y_test)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).